<a href="https://colab.research.google.com/github/FurkyT/Full_Body_Segmentation_Unet/blob/main/segmentation_unet_fullbody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PİP**




In [ ]:
!pip install segmentation_models

**Drive Connect For Download The .py Files To The Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,"/content/drive/MyDrive/ParcelDelineation/models")
import sys
sys.path.insert(0,"/content/drive/MyDrive/ParcelDelineation/utils")
import sys
sys.path.insert(0,"/content/drive/MyDrive/Segment_Model")



Mounted at /content/drive


**Connectin Kaggle For Download The Dataset**

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600  ~/.kaggle/kaggle.json
!kaggle datasets download tapakah68/segmentation-full-body-mads-dataset
!unzip segmentation-full-body-mads-dataset


**Train Cell**

In [ ]:
from keras.preprocessing import image
import segmentation_models as sm
from segmentation_models import Unet
from tensorflow.keras.applications import resnet50, densenet, mobilenet_v2
from keras.models import Model, model_from_json
from keras.layers import Reshape, Concatenate, Conv2D, Conv2DTranspose, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.models import load_model
import keras.losses #import cosine_proximity
from keras import regularizers
from PIL import Image
from random import randint
import unet
import unet_dilated
from data_loader_utils import batch_generator, batch_generator_DG
import metrics
from metrics import f1
import numpy as np
import pandas as pd
import glob
import math
import warnings
import keras.backend as K
import keras
import pdb
import tensorflow as tf
import cv2
import sys

sm.set_framework('tf.keras')
sm.framework()

assert len(sys.argv)==3
prefix = sys.argv[1]
epoch = 10

Image.MAX_IMAGE_PIXELS = None
warnings.simplefilter('ignore', Image.DecompressionBombWarning)

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

def learning_rate_scheduler(epoch):
    lr = 1e-4
    '''
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    '''
    print("Set Learning Rate : {}".format(lr))
    return lr


#Set the variables here for training the model 
is_fill = False
is_stacked = False
is_imageNet = True
is_dilated = False # dilated models are only for non-pretrained models 

num_channels = 3
if is_stacked:
    num_channels = 9

input_shape = (640, 640, num_channels)
batch_size = 12
base_dir = '/content/'
train_file = prefix + 'demoitrain.csv'
val_file = prefix + 'demoival.csv'
filepath = prefix + '-unet-segment.hdf5'
csv_log_file = prefix + '_log_unet_segment.csv'

#Loads training and validation data frame
#Dataframe contains the paths of the images
train_df = pd.read_csv(base_dir + train_file)
val_df = pd.read_csv(base_dir + val_file)

model = None 

if is_dilated:
    model = unet_dilated(input_size = input_shape)
elif is_imageNet:
    model_unet = Unet(BACKBONE, encoder_weights='imagenet')
    if is_stacked: 
        new_model = keras.models.Sequential()
        new_model.add(Conv2D(3, (1,1), padding='same', activation='relu', input_shape=input_shape))
        new_model.add(model_unet)
        model = new_model
    else:
        model = model_unet
else:
    model = unet(input_size=input_shape)


model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=learning_rate_scheduler(0)),
              metrics=['acc', f1])

checkpoint = ModelCheckpoint(filepath, monitor='f1', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger(csv_log_file, append=True, separator=';')
callbacks_list = [checkpoint, csv_logger]

model.fit_generator(batch_generator_DG(train_df, batch_size, is_imageNet), steps_per_epoch=round((len(train_df))/batch_size),
        epochs=int(epoch), validation_data=batch_generator_DG(val_df, batch_size, is_imageNet), validation_steps=round((len(val_df))/batch_size),callbacks=callbacks_list)



**Predict Cell For Test Data**

In [ ]:

import sys
import os
import segmentation_models as sm
from keras.preprocessing import image
from tensorflow.keras.applications import resnet50, densenet, mobilenet_v2
from keras.models import Model, model_from_json
from keras.layers import Reshape, Concatenate, Conv2D, Conv2DTranspose, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.models import load_model
import keras.losses #import cosine_proximity
from keras import regularizers
from PIL import Image
import unet_dilated
from segmentation_models import Unet
from random import randint
import numpy as np
import pandas as pd
import glob
import math
import warnings
import keras.backend as K
import pdb
import tensorflow as tf
from keras.models import load_model
import keras.losses
from matplotlib import pyplot as plt
from metrics import get_metrics, f1, dice_coef_sim
from data_loader_utils import batch_generator, batch_generator_DG, read_imgs_keraspp

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

def learning_rate_scheduler(epoch):
    lr = 1e-4
    '''
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    '''
    print("Set Learning Rate : {}".format(lr))
    return lr

#Set the filepaths here for laoding in the file 
is_fill = False
is_stacked = False
is_imageNet = True
is_dilated = False
image_type = 'sentinel' 


batch_size = 1
num_channels = 3
if is_stacked:
    num_channels = 9


base_dir = '/content/'
val_file = '-fdemoival' 
filepath= '/content/drive/MyDrive/Segment_Model/-f-unet-segment' 
csv_log_file = '/content/drive/MyDrive/Segment_Model/-f_log_unet_segment.csv'

sub_fill = ''
if is_fill:
    sub_fill = '_fill'

#Modify file path depending on fill/boundary task
val_file = val_file + sub_fill + '.csv'
# File path for the model
filepath = filepath + sub_fill + '.hdf5'
# Csv log file
csv_log_file = csv_log_file + sub_fill + '.csv'

#Loads validation data frame
test_df = pd.read_csv(base_dir + val_file)

pred_file= "ci_predictions.npy" 
dependencies = {'f1':f1}

model = load_model(filepath, custom_objects=dependencies)

history = model.predict_generator(batch_generator_DG(test_df, batch_size, is_imageNet), steps = round(len(test_df)/batch_size))
history = history.squeeze()
np.save(pred_file, history)

predictions = np.load(pred_file) 
x_true, y_true = read_imgs_keraspp(test_df)
y_true = y_true.flatten()
y_pred = predictions.flatten()

get_metrics(y_true, y_pred, binarized=False)


**Making CSV For Data Paths and Dataframe**

In [ ]:
import csv
import numpy as np
import pandas as pd


def write_to_csv(pairs, file_name_split_csv, header =['image','mask']):
  with open(file_name_split_csv, 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(header)
    for pair in pairs:
      row0 = pair[0]
      row1 = pair[1]
      rows = [row0, row1]
      csv_writer.writerow(rows)

base = '/content/segmentation_full_body_mads_dataset_1192_img/'
images = 'images/'
masks = 'masks/'


ipath = base + images
mpath = base + masks

df = pd.read_csv("images_t.txt")
df.columns = ["path"]
df2 = pd.read_csv("masks_t.txt")
df2.columns = ["path"]


imfiles = []
mafiles = []

for i in df.path:
  imfiles.append("/content/segmentation_full_body_mads_dataset_1192_img/images/" + i)
for i in df2.path:
  mafiles.append("/content/segmentation_full_body_mads_dataset_1192_img/masks/" + i)



impairs = []


for f in imfiles:
    for f_ in mafiles:
        if f.split('/')[-1] == f_.split('/')[-1]:
            impairs.append((f, f_))
   

np.random.shuffle(impairs)


split = len(impairs)//10*9
imtrain = impairs[:split]
imval = impairs[split:]


print('İmage train size: {}'.format(len(imtrain)))
print('İmage val size: {}'.format(len(imval)))


imtrain_csv = '-fdemoitrain.csv'
imval_csv = '-fdemoival.csv'


write_to_csv(imtrain, imtrain_csv)
write_to_csv(imval, imval_csv)



**Saving The Paths Into a .txt File For Read With Pandas**

In [ ]:
! ls /content/segmentation_full_body_mads_dataset_1192_img/images > /content/images_t.txt
! ls /content/segmentation_full_body_mads_dataset_1192_img/masks > /content/masks_t.txt

**Displaying The Test Prediction Data**

In [ ]:
import numpy
from PIL import Image as im

images = numpy.load('ci_predictions.npy')
from matplotlib import pyplot as plt
plt.imshow(images[0], interpolation='nearest')
plt.show()

In [ ]:
import numpy
from PIL import Image as im
import cv2
from google.colab.patches import cv2_imshow
ima = []
for i in range(len(images)):
  img = images[i]
  img[img<0.5] = 0
  img[img != 0] = 255
  ima.append(img)

#img = images[0]
#img[img<0.5] = 0
#img[img != 0] = 255
cv2_imshow(ima[7])

**Function For Giving A Random Image To The Model For Prediction**

In [ ]:
def get_predict(test):
  n_img = Image.open(test)
  n_img = n_img.resize((640,640)).convert('RGB')
  n_img = np.expand_dims(n_img, axis=0)
  n_img = np.array(n_img)
  load_model('/content/drive/MyDrive/Segment_Model/-f-unet-segment.hdf5', custom_objects=dependencies)
  pred = model.predict(n_img,
                batch_size=None,
                verbose='auto',
                steps=None,
                callbacks=None,
                max_queue_size=10,
                workers=1,
                use_multiprocessing=True)
  return pred

In [ ]:
n_img = get_predict('/content/test0.png') # Image path for external data
n_img = n_img.reshape(640,640,)
n_img[n_img<0.5] = 0
n_img[n_img != 0] = 255
cv2_imshow(n_img)